In [2]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager


In [3]:
# 1. Setup Selenium (Headless mode so it doesn't pop up a window)
chrome_options = Options()
# chrome_options.add_argument("--headless") 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Base URL for UTA Engineering
base_url = "https://www.uta.edu/academics/schools-colleges/engineering/academics/departments/civil/faculty-directory"

# 2. Get the main list of professors (Requests is fine for this part if the links are static)
# However, since we have Selenium running, let's use it for everything to be safe.
driver.get(base_url)
time.sleep(2) # Wait for page to load

soup_main = BeautifulSoup(driver.page_source, "html.parser")
list_of_profs_urls = []
list_of_prof_classes = []

In [4]:
for a in soup_main.find_all('a', href=True):
    if "username" in a["href"]:
        list_of_profs_urls.append(a['href'])

In [5]:
if not list_of_profs_urls:
    print("No professor URLs found. Check the selector.")
else:
    for target_url in list_of_profs_urls:
        # 3. Go to the specific professor's page
        print(f"Scraping: {target_url}")
        
        driver.get(target_url+"#Teaching")
        
        # CRITICAL: Wait for JavaScript to load the "Teaching" section
        # Sometimes you might need to click the tab button if it's hidden behind a click
        time.sleep(5) 

        # 4. Pass the rendered HTML to BeautifulSoup
        soup_profile = BeautifulSoup(driver.page_source, "html.parser")

        target_semester = "Spring 2026"
        semester_header = soup_profile.find(
        "h3", 
        class_="dm-profile-heading", 
        string=lambda text: text and target_semester in text
        )
        # 5. Find the teaching items
        
        if semester_header:
            teaching_items = soup_profile.find_all('ol', class_="dm-profile-activities content--heading8--37285")
            try:
                if not teaching_items:
                    print("No teaching items found. The content might be inside an iframe or requires a click.")
                    list_of_prof_classes.append(teaching_items[0].text.split("."))
                else:
                    list_of_prof_classes.append(teaching_items[0].text.split("."))
                    print("Added classes")
            except:
                print("Likely a Emeritus Professor")
        else:
            print("Does not offer Spring 2026 Classes")
# Close the browser
print(list_of_prof_classes)
# driver.quit()

Scraping: https://www.uta.edu/academics/faculty/profile?username=hchoi
Added classes
Scraping: https://www.uta.edu/academics/faculty/profile?username=chenx
Added classes
Scraping: https://www.uta.edu/academics/faculty/profile?username=danoglidisp
Does not offer Spring 2026 Classes
Scraping: https://www.uta.edu/academics/faculty/profile?username=eismaj
Added classes
Scraping: https://www.uta.edu/academics/faculty/profile?username=nickfang
Added classes
Scraping: https://www.uta.edu/academics/faculty/profile?username=hams
Added classes
Scraping: https://www.uta.edu/academics/faculty/profile?username=hossain
Does not offer Spring 2026 Classes
Scraping: https://www.uta.edu/academics/faculty/profile?username=hoyos
Added classes
Scraping: https://www.uta.edu/academics/faculty/profile?username=hummelm
Added classes
Scraping: https://www.uta.edu/academics/faculty/profile?username=hyunkk
Does not offer Spring 2026 Classes
Scraping: https://www.uta.edu/academics/faculty/profile?username=jalalihh

In [6]:
import pandas as pd

In [7]:
ls = pd.DataFrame(list_of_profs_urls)

In [ ]:
for i in list_of_prof_classes:
    for j in i:
        if "CE" in j:
            print(j.split(","))
    print("\n")

['CE 3131', ' section 001', ' ENVIRONMENTAL ANALYSIS']
['CE 3131', ' section 002', ' ENVIRONMENTAL ANALYSIS']
['CE 3131', ' section 003', ' ENVIRONMENTAL ANALYSIS']
['CE 5319', ' section 001', ' PHYSICAL-CHEMICAL PROCESSES II']
['CE 4353', ' section 101', ' WATER CHEMISTRY']


['CE 5325', ' section 001', ' BIOLOGICAL PROCESSES']
['CE 5325', ' section 101', ' BIOLOGICAL PROCESSES']
['CE 4357', ' section 001', ' INTRO BIO WASTEWATER TREATMENT']
['CE 4357', ' section 101', ' INTRO BIO WASTEWATER TREATMENT']


['CE 3342', ' section 001', ' WATER RESOURCES ENGINEERING']
['CE 3342', ' section 101', ' WATER RESOURCES ENGINEERING']


['CE 4328', ' section 001', ' WATER SYSTEM DESIGN']


['CE 3341', ' section 001', ' STRUCTURAL ANALYSIS']
['CE 3341', ' section 101', ' STRUCTURAL ANALYSIS']


['CE 5367', ' section 101', ' DESIGN OF EARTH STRUCTURES']
['CE 4320', ' section 001', ' EARTH STRUCTURES DESIGN']
['CE 4320', ' section 101', ' EARTH STRUCTURES DESIGN']
['CE 3343', ' section 001', ' SOIL 